# maude-magic

> Fill in a module description here

**Refs:**
- [pexpect](https://pexpect.readthedocs.io/en/stable/)
- [nbdev](https://nbdev.fast.ai/tutorials/tutorial.html)

In [32]:
#| default_exp maude-magic

In [33]:
#| hide
from nbdev.showdoc import *

In [34]:
import os
import pexpect

In [61]:
#| export
timeout = 2

class MaudeInterpreter:
    """Controls maude execution, executing commands and print responses.
       Preserve sessions between different cell executions."""
    def __init__(self,debug=False,timeout=timeout):
        """ Init variables and spawns maude. """
        self.debug,self.timeout = debug,timeout
        self._prompt='Maude>'
        # Execute maude in environment "env". Add current working directory to MAUDE_LIB
        env = dict(os.environ)
        if not 'MAUDE_LIB' in env: raise(Exception('MAUDE_LIB environmet variable not found'))
        env['MAUDE_LIB'] += ':' + os.getcwd()
        if self.debug: 
            print(f"MAUDE_LIB={env['MAUDE_LIB']}")
        
        # the expawned process:
        self.sh = pexpect.spawn('maude', encoding='utf-8', env=env)
        self.sh.expect(self._prompt,timeout=timeout)
        if self.debug: print(self.sh.before)

    def __call__(self,command,timeout=timeout):
        self.sh.sendline(command)
        if self.debug: print('command:\n'+command)
        i = self.sh.expect([pexpect.EOF,pexpect.TIMEOUT,self._prompt],timeout)
        if i == 0:
            if self.debug: print('Normally terminated')
            raise(Exception('EOF'))    
        elif i == 1:
            if self.debug: print('Timeout')
            raise(Exception('Timeout'))    
        else:
            response = self.sh.before
        if self.debug: print('response:\n'+response)
        return response
        

Current directory must be in the spawned process environment:

In [62]:
maude=MaudeInterpreter(debug=True)

MAUDE_LIB=/usr/local/lib/Maude-3.5.1-linux-x86_64/:/home/usuario/maude-magic/nbs
		     \||||||||||||||||||/
		   --- Welcome to Maude ---
		     /||||||||||||||||||\
	    Maude 3.5.1 built: Jul 16 2025 12:00:00
	     Copyright 1997-2025 SRI International
		   Mon Dec 29 01:07:17 2025



In [63]:
maude('quit .')

command:
quit .
Normally terminated


Exception: EOF

In [37]:
#| export
def say_hello(to):
    print(f"Hello, {to}!")

In [25]:
#| hide
import nbdev; nbdev.nbdev_export()